## This file implements translation evaluations using a modified GEMBA prompt system for the dev dataset.

### Install Libraries

In [1]:
%pip install pandas
%pip install langchain
%pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Imports

In [2]:
import os
import ast
import pandas as pd
from tqdm import tqdm

# Langchain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

### Globals

In [3]:
LLM_MODEL_NAME="gpt-3.5-turbo"

### Utility function to set up the OpenAI API

In [4]:
def load_openai_key():
    try:
        # Open and read the entire content of the file
        with open("openai-key.txt", 'r') as file:
            contents = file.read()
        
        return contents
    
    except FileNotFoundError:
        print(f"The file does not exist.")
        return None
    except Exception as e:
        # Handle other potential exceptions (e.g., permission errors)
        print(f"An error occurred while reading the file: {str(e)}")
        return None

openai_key = load_openai_key()
os.environ["OPENAI_API_KEY"] = openai_key

chat = ChatOpenAI(model="gpt-3.5-turbo")

### Prompt definition

In [5]:
system = """\
You are a helpful evaluator of the quality of translations.
Score the following translations from English to Portuguese on a continuous scale from 0 to 100, 
where a score of zero means "no meaning preserved" and score of one hundred means
"perfect meaning and grammar".
"""

human = """\
En: {source_seg}
Pt 1: "{target_seg1}" 
Pt 2: "{target_seg2}"
Pt 3: "{target_seg3}"
Reply with only the number scores of your evaluation, in a python list:
"""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

### Initial test

In [6]:
chain = prompt | chat
response = chain.invoke(input={"source_seg": "There are more things between heaven and earth",
                    "target_seg1": "Existem mais coisas entre o céu e a terra",
                    "target_seg2": "Existem coisas entre o céu e a terra",
                    "target_seg3": "Existem coisas entre o paraíso e a terra"})

In [7]:
print(response.content)

[80, 70, 60]


In [8]:
print(response.response_metadata.get('token_usage').get('total_tokens'))

155


In [9]:
print(response.content)

[80, 70, 60]


In [10]:
actual_list = ast.literal_eval(response.content)

In [11]:
actual_list[0]

80

### Translation Datasets for comparisons (DEV)

In [12]:
df_dev = pd.read_csv('data/dev.csv')
df_dev = df_dev.drop(columns=['Unnamed: 0'])

In [13]:
df_dev

,context,question,answerA,answerB,answerC,correct
0,Tracy didn't go home that evening and resisted...,What does Tracy need to do before this?,make a new plan,Go home and see Riley,Find somewhere to go,C
1,Sydney walked past a homeless woman asking for...,How would you describe Sydney?,sympathetic,like a person who was unable to help,incredulous,A
2,Sasha protected the patients' rights by making...,What will patients want to do next?,write new laws,get petitions signed,live longer,B
3,Jordan was in charge of taking the food on the...,How would Jordan feel afterwards?,horrible that he let his friends down on the c...,happy that he doesn't need to do the cooking o...,very proud and accomplished about the camping ...,A
4,Kendall opened their mouth to speak and what c...,How would you describe Kendall?,a very quiet person,a very passive person,a very aggressive and talkative person,C
...,...,...,...,...,...,...
1949,Bailey destroyed the enemy's bunker and won th...,What does Bailey need to do before this?,spy on enemy troop movements,give the enemy counterintelligence,win the war for his country,A
1950,Kai told Kendall the reason about the accident...,How would you describe Kai?,more at ease,like they got something off their chest,feeling sad,B
1951,Kai improved Jan's picture and she loved how i...,What will Jan want to do next?,wanted to be helpful,wanted to Jan to be glad,frame the picture,C
1952,Tracy's kids wanted ice cream so Aubrey fed th...,What does Aubrey need to do before this?,tell her kids to say thank you,get ice cream,thanks Aubrey,B


In [14]:
df_dev1_pt = pd.read_csv('translated/marian_mt/dev_pt.csv')
df_dev2_pt = pd.read_csv('translated/unicamp_t5/dev_pt.csv')
df_dev3_pt = pd.read_csv('translated/nllb_1.3b/dev_pt.csv')

# clean up data
df_dev1_pt = df_dev1_pt.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
df_dev2_pt = df_dev2_pt.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'correct'])
df_dev3_pt = df_dev3_pt.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'correct'])

# Rename columns
df_dev1_pt = df_dev1_pt.rename(columns={'context': 'context_1', 'question': 'question_1',
                                  'answerA': 'answerA_1', 'answerB': 'answerB_1',
                                  'answerC': 'answerC_1'})
df_dev2_pt = df_dev2_pt.rename(columns={'context': 'context_2', 'question': 'question_2',
                                  'answerA': 'answerA_2', 'answerB': 'answerB_2',
                                  'answerC': 'answerC_2'})
df_dev3_pt = df_dev3_pt.rename(columns={'context': 'context_3', 'question': 'question_3',
                                  'answerA': 'answerA_3', 'answerB': 'answerB_3',
                                  'answerC': 'answerC_3'})
# Merge the three dataframes
df_dev_pt = pd.concat((df_dev1_pt, df_dev2_pt, df_dev3_pt), axis=1)
df_dev_pt = df_dev_pt[['context_1', 'context_2', 'context_3',
                 'question_1', 'question_2', 'question_3',
                 'answerA_1', 'answerA_2', 'answerA_3',
                 'answerB_1', 'answerB_2', 'answerB_3',
                 'answerC_1', 'answerC_2', 'answerC_3',
                 'correct']]

In [15]:
df_dev_pt

,context_1,context_2,context_3,question_1,question_2,question_3,answerA_1,answerA_2,answerA_3,answerB_1,answerB_2,answerB_3,answerC_1,answerC_2,answerC_3,correct
0,Tracy não foi para casa naquela noite e resist...,Tracy não ia para casa naquela noite e resisti...,A Tracy não foi para casa naquela noite e resi...,O que Tracy precisa fazer antes disso?,O que é necessário para Tracy antes disso?,O que é que a Tracy precisa de fazer antes disto?,Faça um novo plano,elaborar um novo plano,Faça um novo plano,Vá para casa e veja Riley,Ir para casa e ver Riley,Vai para casa ver a Riley.,Encontre um lugar para ir,Encontrar em algum lugar para ir,Encontrar um sítio para ir,C
1,Sydney passou por uma mulher sem-teto pedindo ...,Sydney passou uma mulher em situação de rua pe...,Sydney passou por uma sem-abrigo pedindo troco...,Como você descreveria Sydney?,Como descreveria Sydney?,Como descreveria Sydney?,simpático,simpática,simpático,como uma pessoa que não foi capaz de ajudar,como uma pessoa que não conseguiu ajudar,Como uma pessoa incapaz de ajudar,incrédulo,inchaço,incrédulos.,A
2,"Sasha protegeu os direitos dos pacientes, faze...",Sasha protegeu os direitos dos pacientes ao fa...,Sasha protegeu os direitos dos pacientes fazen...,O que os pacientes vão querer fazer a seguir?,O que os pacientes querem fazer a seguir?,O que os pacientes vão querer fazer a seguir?,Escrever novas leis,redigidos novos diplomas,Escrever novas leis,obter petições assinadas,au,Obter petições assinadas,viver mais tempo,vida,viver mais tempo,B
3,Jordan estava encarregado de levar a comida na...,Jordan foi responsável por levar a comida na v...,A Jordan era responsável por levar a comida pa...,Como Jordan se sentiria depois?,Como a Jordânia se sentiria depois?,Como se sentiria Jordan depois disso?,horrível que ele deixou seus amigos para baixo...,horrível que ele deixe seus amigos para baixo ...,É horrível que ele tenha decepcionado os amigo...,feliz que ele não precisa fazer o cozimento na...,feliz que ele não precisa fazer o cozimento na...,Feliz por não ter de cozinhar na viagem.,muito orgulhoso e realizado sobre a viagem de ...,muito orgulho e realizado com relação à viagem...,muito orgulhosa e feliz com a viagem de acampa...,A
4,Kendall abriu a boca para falar e o que saiu c...,Kendall abriu a boca para falar e o que saiu c...,Kendall abriu a boca para falar e o que saiu c...,Como você descreveria Kendall?,Como descrever Kendall?,Como descreveria a Kendall?,uma pessoa muito tranquila,uma pessoa muito silenciosa,Uma pessoa muito quieta.,Uma pessoa muito passiva,uma pessoa muito passiva,Uma pessoa muito passiva.,uma pessoa muito agressiva e falante,uma pessoa muito agressiva e falante,uma pessoa muito agressiva e faladora,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1949,Bailey destruiu o bunker do inimigo e ganhou a...,Bailey destruiu o bunker do inimigo e venceu a...,Bailey destruiu o bunker do inimigo e ganhou a...,O que Bailey precisa fazer antes disso?,O que é necessário para Bailey antes disso?,O que é que a Bailey precisa de fazer antes di...,espionar os movimentos das tropas inimigas,esperam os movimentos de tropas inimigas,Espião dos movimentos das tropas inimigas,Dê ao inimigo contra-inteligência,da contra-inteligência dos inimigos,Dar ao inimigo contra-inteligência,Vencer a guerra pelo seu país,vencer a guerra pelo seu país,ganhar a guerra pelo seu país.,A
1950,Kai contou a Kendall a razão do acidente no la...,Kai contou o motivo do acidente no lago no últ...,O Kai disse à Kendall o motivo do acidente no ...,Como você descreveria Kai?,Como descreveria Kai?,Como descreveria o Kai?,Mais à vontade,mais à vontade,mais à vontade .,Como se tivessem tirado algo do peito,como eles tiraram algo do peito,Como se tivessem algo no peito.,Sentindo-se triste,sentir- se triste,sentir-se triste,B
1951,Kai melhorou a imagem de Jan e ela adorou como...,Kai melhorou a imagem de Jan e ela amava como ...,O Kai melhorou a foto da Jan e ela adorou 

In [16]:
def create_sequences(i):

        sentence_list = ['context', 'question', 'answerA', 'answerB', 'answerC']
        english_str=""
        for sentence in sentence_list:
                english_str += str(df_dev.loc[i, sentence])
                english_str += "\n"

        portuguese_1=""
        sentence_list = ['context_1', 'question_1', 'answerA_1', 'answerB_1', 'answerC_1']
        for sentence in sentence_list:
                portuguese_1 += str(df_dev_pt.loc[i, sentence])
                portuguese_1 += "\n"

        portuguese_2=""
        sentence_list = ['context_2', 'question_2', 'answerA_2', 'answerB_2', 'answerC_2']
        for sentence in sentence_list:
                portuguese_2 += str(df_dev_pt.loc[i, sentence])
                portuguese_2 += "\n"

        portuguese_3=""
        sentence_list = ['context_3', 'question_3', 'answerA_3', 'answerB_3', 'answerC_3']
        for sentence in sentence_list:
                portuguese_3 += str(df_dev_pt.loc[i, sentence])
                portuguese_3 += "\n"

        return english_str, portuguese_1, portuguese_2, portuguese_3

In [17]:
def get_best_translations(df_src_lang, df_tgt_lang):

    total_tokens = 0

    df_output = df_src_lang.copy()
    df_output.loc[:, ['context', 'question', 'answerA', 'answerB', 'answerC']] = None
    df_rankings = pd.DataFrame(columns=['MarianMT', 'Unicamp_T5', 'NLLB_1.3b'])

    for i in tqdm(range(len(df_src_lang))):

        # OpenAI call
        chain = prompt | chat
        english, translation_pt1, translation_pt2, translation_pt3 = create_sequences(i)
        response = chain.invoke(input={"source_seg": english,
                        "target_seg1": translation_pt1,
                        "target_seg2": translation_pt2,
                        "target_seg3": translation_pt3})
        
        total_tokens += response.response_metadata.get('token_usage').get('total_tokens')

        results = ast.literal_eval(response.content)            
        best = results.index(max(results))
        
        # Add to rankings dataset
        df_rankings.loc[len(df_rankings)] = results
        
        # Copy to final dataset
        col_list = ['context', 'question', 'answerA', 'answerB', 'answerC']
        for col in col_list:
            col_best = col + '_' + str(best+1)
            best_result = df_tgt_lang.loc[i, col_best]
            sentence_idx = col_list.index(col)
            df_output.iat[i, sentence_idx] = best_result

    print(f"Total tokens used: {total_tokens}")
    
    return df_output, df_rankings

In [18]:
df_dev.loc[0]

context     Tracy didn't go home that evening and resisted...
question              What does Tracy need to do before this?
answerA                                       make a new plan
answerB                                 Go home and see Riley
answerC                                  Find somewhere to go
correct                                                     C
Name: 0, dtype: object

In [19]:
# Test base
df_dev_final, df_dev_rankings = get_best_translations(df_dev, df_dev_pt)

  0%|          | 0/1954 [00:00<?, ?it/s]

100%|██████████| 1954/1954 [20:35<00:00,  1.58it/s]

Total tokens used: 602042


In [24]:
df_dev_final

,context,question,answerA,answerB,answerC,correct
0,Tracy não foi para casa naquela noite e resist...,O que Tracy precisa fazer antes disso?,Faça um novo plano,Vá para casa e veja Riley,Encontre um lugar para ir,C
1,Sydney passou por uma mulher sem-teto pedindo ...,Como você descreveria Sydney?,simpático,como uma pessoa que não foi capaz de ajudar,incrédulo,A
2,Sasha protegeu os direitos dos pacientes fazen...,O que os pacientes vão querer fazer a seguir?,Escrever novas leis,Obter petições assinadas,viver mais tempo,B
3,A Jordan era responsável por levar a comida pa...,Como se sentiria Jordan depois disso?,É horrível que ele tenha decepcionado os amigo...,Feliz por não ter de cozinhar na viagem.,muito orgulhosa e feliz com a viagem de acampa...,A
4,Kendall abriu a boca para falar e o que saiu c...,Como descreveria a Kendall?,Uma pessoa muito quieta.,Uma pessoa muito passiva.,uma pessoa muito agressiva e faladora,C
...,...,...,...,...,...,...
1949,Bailey destruiu o bunker do inimigo e ganhou a...,O que é que a Bailey precisa de fazer antes di...,Espião dos movimentos das tropas inimigas,Dar ao inimigo contra-inteligência,ganhar a guerra pelo seu país.,A
1950,Kai contou o motivo do acidente no lago no últ...,Como descreveria Kai?,mais à vontade,como eles tiraram algo do peito,sentir- se triste,B
1951,Kai melhorou a imagem de Jan e ela adorou como...,O que Jan vai querer fazer a seguir?,queria ser útil,queria que Jan fosse feliz,enquadrar a imagem,C
1952,"Os filhos da Tracy queriam gelado, por isso o ...",O que é que o Aubrey precisa de fazer antes di...,Diz aos filhos que lhe digam obrigado .,- Vamos comer gelado.,"Obrigado , Aubrey .",B


In [25]:
df_dev_rankings

,MarianMT,Unicamp_T5,NLLB_1.3b
0,85,75,80
1,90,85,80
2,95,70,100
3,85,80,90
4,75,70,80
...,...,...,...
1949,85,70,90
1950,85,90,80
1951,85,60,75
1952,65,50,75


In [26]:
df_dev_final.to_csv('translated/final/dev.csv')
df_dev_rankings.to_csv('rankings/dev_rankings.csv')